# 2교시 기본 연산자 다루기

> 스파크의 "기본 연산자" 와 "데이터프레임"에 대해 학습합니다

## 목차
* [1. 기본 연산자](#1.-기본-연산자)
  - [1.1 데이터 프레임 함수](#1.1-데이터-프레임-x함수)
  - [1.2 컬럼 함수](#1.2-컬럼-함수)
  - [1.3 기타 함수](#1.3-기타-함수)
* [2. RDD 의 특징](#3.-RDD-의-특징)
  - [2.1 RDD 통한 데이터 변환](#3.1-RDD-통한-데이터-변환)
  - [2.2 구조화 API 통한 데이터 변환](#3.2-구조화-API-통한-데이터-변환)
* [3. 데이터 타입](#4.-데이터-타입)
* [4. 핵심 데이터 프레임 연산자](#2.-핵심-데이터-프레임-연산자)
  - [4.1 파일로 부터 테이블 만들어 사용하기](#2.1-파일로-부터-테이블-만들어-사용하기)
  - [4.2 특정 컬럼 선택 (select, selectExpr)](#2.2-특정-컬럼-선택-(select,-selectExpr))
  - [4.3 상수값 사용하기](#2.3-상수값-사용하기)
  - [4.4 컬럼 추가하기](#2.4-컬럼-추가하기)
  - [4.5 컬럼명 바꾸기](#2.5-컬럼명-바꾸기)
  - [4.6 컬럼 제거하기](#2.6-컬럼-제거하기)
  - [4.7 컬럼의 데이터 타입 변경하기](#2.7-컬럼의-데이터-타입-변경하기)
  - [4.8 레코드 필터링](#2.8-레코드-필터링)
  - [4.9 유일값 (DISTINCT)](#2-9-유일값-(DISTINCT))
  - [4.10 정렬 (SORT)](#2.10-정렬-(SORT))
  - [4.11 로우 수 제한 (LIMIT)](#2.11-로우-수-제한-(LIMIT))
* [5. 기타 데이터 프레임 연산자](#5.-기타-데이터-프레임-연산자)
  - [5.1 사전에 스키마를 정의하는 장점](#5.1-사전에-스키마를-정의하는-장점)
  - [5.2 스키마를 정의하는 두 가지 방법](#5.2-스키마를-정의하는-두-가지-방법)
  - [5.3 중첩된 배열 스키마](#5.3-중첩된-배열-스키마)
  - [5.4 컬럼과 표현식](#5.4-컬럼과-표현식)
  - [5.5 로우 생성 및 다루기](#5.5-로우-생성-및-다루기)
  - [5.6 파케이 파일 혹은 테이블 저장](#5.6-파케이-파일-혹은-테이블-저장)
  - [5.7 프로젝션과 필터](#5.7-프로젝션과-필터)
  - [5.8 날짜 관련 함수](#5.8-날짜-관련-함수)
* [6. 데이터셋 API](#6.-데이터셋-API)
  - [6.1 데이터셋과 데이터프레임 비교](#6.1-데이터셋과-데이터프레임-비교)
  - [6.2 데이터셋 데이터프레임 그리고 RDD](#6.2-데이터셋-데이터프레임-그리고-RDD)
* [7. 카탈리스트 옵티마이저](#7.-카탈리스트-옵티마이저)
  - [7.1 분석 (Analysis)](#7.1-분석-(Analysis))
  - [7.2 논리 최적화 (Logical Optimization)](#7.2-논리-최적화-(Logical-Optimization))
  - [7.3 물리 계획 (Physical Planning)](#7.3-물리-계획-(Physical-Planning))
  - [7.4 코드 생성 (Code Generation)](#7.4-코드-생성-(Code-Generation))
* [8. 실습 문제](#8.-실습-문제)
* [참고자료](#참고자료)

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)
# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

## 1. 기본 연산자
---
### 1.1 데이터 프레임 함수
| 함수 | 설명 | 기타 |
| - | - | - |
| df.printSchema() | 스키마 정보를 출력합니다. | - |
| df.schema | StructType 스키마를 반환합니다 | - |
| df.columns | 컬럼명 정보를 반환합니다 | - |
| df.show(n) | 데이터 n 개를 출력합니다 | - |
| df.first() | 데이터 프레임의 첫 번째 Row 를 반환합니다 | - |
| df.head(n) | 데이터 프레임의 처음부터 n 개의 Row 를 반환합니다 | - |
| df.createOrReplaceTempView | 임시 뷰 테이블을 생성합니다 | - |
| df.union(newdf) | 데이터프레임 간의 유니온 연산을 수행합니다 | - |
| df.limit(n) | 추출할 로우수 제한 | T |
| df.repartition(n) | 파티션 재분배, 셔플발생 | - |
| df.coalesce() | 셔플하지 않고 파티션을 병합 | 마지막 스테이지의 reduce 수가 줄어드는 효과로 성능저하에 유의해야 합니다 |
| df.collect() | 모든 데이터 수집, 반환 | A |
| df.take(n) | 상위 n개 로우 반환 | A |

---
### 1.2 컬럼 함수
| 함수 | 설명 | 기타 |
| - | - | - |
| df.select | 컬럼이나 표현식 사용  | - |
| df.selectExpr | 문자열 표현식 사용 = df.select(expr()) | - |
| df.withColumn(컬럼명, 표현식) | 컬럼 추가, 비교, 컬럼명 변경 | - |
| df.withColumnRenamed(old_name, new_name) | 컬럼명 변경 | - |
| df.drop() | 컬럼 삭제 | - |
| df.where | 로우 필터링 | - |
| df.filter | 로우 필터링 | - |
| df.sort, df.orderBy | 정렬 | - |
| df.sortWithinPartitions | 파티션별 정렬 | - |

---
### 1.3 기타 함수
| 함수 | 설명 | 기타 |
| - | - | - |
| expr("someCol - 5") | 표현식 | - |
| lit() | 리터럴 | - |
| cast() | 컬럼 데이터 타입 변경 | - |
| distinct() | unique row | - |
| desc(), asc() | 정렬 순서 | - |


## 2. RDD 의 특징

| 특징 | 설명 | 기타 |
|---|---|---|
| dependencies | resilency | 리니지를 통해 의존성 정보를 유지함으로써 언제든 다시 수행할 수 있는 회복력을 가집니다 |
| partitions | parallelize computation | 파티션 단위로 데이터를 저장 관리하므로써 병렬 처리를 가능하게 합니다 |
| compute function | Iterator\[T\] | RDD로 저장되는 모든 데이터는 반복자를 통해 함수를 적용할 수 있습니다 |

* 반면에 compute function 의 내부를 spark 가 알 수 없기 때문에 오류를 찾아내가 어려우며, Python 과 같은 스크립트 언어는 generic object 로만 인식이 되므로 호환하기 어려우며, T 타입의 객체는 직렬화되어 전달되기만 할 뿐 스파크는 해당 데이터 타입 T 에 대해 알 수 없습니다

> RDD 를 통해 데이터 처리하는 방법과, 구조화된 API 를 통해 처리하는 방법을 비교해 보고, 이러한 고수준의 DSL 연산자를 통해 보다 단순하게 표현이 가능합니다.

### 2.1 RDD 통한 데이터 변환 (참고용)

In [2]:
dataRDD = spark.sparkContext.parallelize([("Cat", 30), ("Dog", 28), ("Monkey", 28), ("Cat", 24), ("Dog", 10)])
agesRDD = (
    dataRDD.map(lambda x: (x[0], (x[1], 1)))
     .reduceByKey(lambda v1, v2: (v1[0] + v2[0], v1[1] + v2[1]))
     .map(lambda v: (v[0], v[1][0]/v[1][1]))
)
agesRDD.toDF(["Name", "Age"]).show()

+------+----+
|  Name| Age|
+------+----+
|   Cat|27.0|
|   Dog|19.0|
|Monkey|28.0|
+------+----+



### 2.2 구조화 API 통한 데이터 변환

In [3]:
spark = SparkSession.builder.appName("동물의 평균 수명").getOrCreate()
animal = spark.createDataFrame([("Cat", 30), ("Dog", 28), ("Monkey", 28), ("Cat", 24), ("Dog", 10)], ["Name", "Age"])
ages = animal.select("Name", "Age").groupBy("Name").agg(avg("Age").alias("Age"))
ages.show(truncate=False)

+------+----+
|Name  |Age |
+------+----+
|Cat   |27.0|
|Monkey|28.0|
|Dog   |19.0|
+------+----+



## 3. 데이터 타입
> immutable 하며, 모든 transformation 들의 lineage 를 유지합니다. 또한 컬럼을 변경, 추가 등을 통해 새로운 데이터프레임을 생성합니다.

| python | scala |
|---|---|
| ![python](images/datatypes-python.png) | ![scala](images/datatypes-scala.png) | 
| ![python](images/datatypes-python2.png) | ![scala](images/datatypes-scala2.png) | 


## 4 핵심 데이터 프레임 연산자

### 4.1 파일로 부터 테이블 만들어 사용하기

In [4]:
print("# 원시 데이터로 부터 읽거나, Spark SQL 통한 결과는 항상 데이터프레임이 생성됩니다")
df = spark.read.json("data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("2015_summary")

sql_result = spark.sql("SELECT * FROM 2015_summary").show(5)

# 원시 데이터로 부터 읽거나, Spark SQL 통한 결과는 항상 데이터프레임이 생성됩니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



### 4.2 특정 컬럼 선택 (select, selectExpr)
> 아래의 모든 예제에서 컬럼 선택 시에 select(col("컬럼명")) 으로 접근할 수도 있지만 **selectExpr("컬럼명") 이 간결하기 때문에 앞으로는 가능한 표현식으로 사용**하겠습니다 <br>
컬럼 표현식의 경우 반드시 하나의 컬럼은 하나씩 표현되어야만 합니다.  <br>
잘된예 : "컬럼1", "컬럼2" <br>
잘못된예: "컬럼1, 컬럼2"

In [5]:
from pyspark.sql.functions import *

print("# select 표현은 컬럼만 입력이 가능하며, 함수나 기타 표현식을 사용할 수 없습니다. 사용하기 위해서는 functions 를 임포트 하고, 개별 함수의 특징을 잘 이해하고 사용해야 합니다")
df.select(upper(col("DEST_COUNTRY_NAME")), "ORIGIN_COUNTRY_NAME").show(2)

print("# selectExpr 별도의 임포트 없이, 모든 표현식을 사용할 수 있습니다")
df.selectExpr("upper(DEST_COUNTRY_NAME)", "ORIGIN_COUNTRY_NAME").show(2)

# select 표현은 컬럼만 입력이 가능하며, 함수나 기타 표현식을 사용할 수 없습니다. 사용하기 위해서는 functions 를 임포트 하고, 개별 함수의 특징을 잘 이해하고 사용해야 합니다
+------------------------+-------------------+
|upper(DEST_COUNTRY_NAME)|ORIGIN_COUNTRY_NAME|
+------------------------+-------------------+
|           UNITED STATES|            Romania|
|           UNITED STATES|            Croatia|
+------------------------+-------------------+
only showing top 2 rows

# selectExpr 별도의 임포트 없이, 모든 표현식을 사용할 수 있습니다
+------------------------+-------------------+
|upper(DEST_COUNTRY_NAME)|ORIGIN_COUNTRY_NAME|
+------------------------+-------------------+
|           UNITED STATES|            Romania|
|           UNITED STATES|            Croatia|
+------------------------+-------------------+
only showing top 2 rows



In [6]:
print("# 컬럼의 앨리어스 혹은 전체 컬럼을 위한 * 도 사용할 수 있습니다")
df.selectExpr("DEST_COUNTRY_NAME as newColmnName", "DEST_COUNTRY_NAME").show(2)

df.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry").show(2)

# 컬럼의 앨리어스 혹은 전체 컬럼을 위한 * 도 사용할 수 있습니다
+-------------+-----------------+
| newColmnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



### <font color=green>1. [기본]</font> "data/flight-data/json/2015-summary.json" 경로의 JSON 데이터를 읽고
#### 1. 스키마를 출력하세요
#### 2. 10건의 데이터를 출력하세요
#### 3. selectExpr 구문 혹은 spark sql 구문을 이용하여 DEST_COUNTRY_NAME 컬럼은 대문자로, ORIGIN_COUNTRY_NAME 컬럼을 소문자로 2개의 컬럼을 조회하세요

<details><summary>[실습1] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다


```python
df1 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)
    
df1.printSchema()
answer = df1.createOrReplaceTempView("2015_summary")
spark.sql("select upper(DEST_COUNTRY_NAME), lower(ORIGIN_COUNTRY_NAME) from 2015_summary").show(10)
```

</details>


In [7]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df1 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)

df1.printSchema()
answer = df1.createOrReplaceTempView("2015_summary")
spark.sql("select upper(DEST_COUNTRY_NAME), lower(ORIGIN_COUNTRY_NAME) from 2015_summary").show(10)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+------------------------+--------------------------+
|upper(DEST_COUNTRY_NAME)|lower(ORIGIN_COUNTRY_NAME)|
+------------------------+--------------------------+
|           UNITED STATES|                   romania|
|           UNITED STATES|                   croatia|
|           UNITED STATES|                   ireland|
|                   EGYPT|             united states|
|           UNITED STATES|                     india|
|           UNITED STATES|                 singapore|
|           UNITED STATES|                   grenada|
|              COSTA RICA|             united states|
|                 SENEGAL|             united states|
|                 MOLDOVA|             united states|
+------------------------+--------------------------+
only showing top 10 rows



### 4.3 상수값 사용하기

In [8]:
# 리터럴(literal)을 사용한 리터럴 상수 값 컬럼 추가
from pyspark.sql.functions import lit

# df.select(expr("*"), lit(1).alias("One")).show(2)
df.selectExpr("*", "1 as One").show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



### 4.4 컬럼 추가하기

In [9]:
print("# withColumn(컬럼명, 표현식) 으로 컬럼 추가")
df.withColumn("numberOne", lit(1)).show(2)

# withColumn(컬럼명, 표현식) 으로 컬럼 추가
+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [10]:
print("# 컬럼의 대소 비교를 통한 불리언 값 반환")
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

# 컬럼의 대소 비교를 통한 불리언 값 반환
+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [11]:
print("# 존재하는 컬럼을 표현식을 통해 새로운 컬럼 생성, 기존 컬럼을 삭제")
before = df
before.printSchema()

after = before.withColumn("Destination", expr("DEST_COUNTRY_NAME"))
after.printSchema()

# 존재하는 컬럼을 표현식을 통해 새로운 컬럼 생성, 기존 컬럼을 삭제
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- Destination: string (nullable = true)



### <font color=green>2. [기본]</font> "data/flight-data/json/2015-summary.json" 경로의 JSON 데이터를 읽고
#### 1. 스키마를 출력하세요
#### 2. 10건의 데이터를 출력하세요
#### 3. 기존의 컬럼은 그대로 두고, ORIGIN_COUNTRY_NAME, DEST_COUNTRY_NAME 2개의 컬럼을 각각 소문자, 대문자로 변경한 컬럼 ORIGIN_COUNTRY_NAME_LOWER, DEST_COUNTRY_NAME_UPPER 총 4개의 컬럼을 출력하세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다


```python
df2 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)
    
df2.printSchema()
answer = df2.createOrReplaceTempView("2015_summary")
spark.sql("""select 
    ORIGIN_COUNTRY_NAME, 
    DEST_COUNTRY_NAME, 
    lower(ORIGIN_COUNTRY_NAME) as ORIGIN_COUNTRY_NAME_LOWER, 
    upper(DEST_COUNTRY_NAME) as DEST_COUNTRY_NAME_UPPER 
    from 2015_summary"""
).show(10)
```

</details>


In [12]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df2 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)

df2.printSchema()
answer = df2.createOrReplaceTempView("2015_summary")
spark.sql("""select 
    ORIGIN_COUNTRY_NAME, 
    DEST_COUNTRY_NAME, 
    lower(ORIGIN_COUNTRY_NAME) as ORIGIN_COUNTRY_NAME_LOWER, 
    upper(DEST_COUNTRY_NAME) as DEST_COUNTRY_NAME_UPPER 
    from 2015_summary"""
).show(10)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+-------------------+-----------------+-------------------------+-----------------------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME_LOWER|DEST_COUNTRY_NAME_UPPER|
+-------------------+-----------------+-------------------------+-----------------------+
|            Romania|    United States|                  romania|          UNITED STATES|
|            Croatia|    United States|                  croatia|          UNITED STATES|
|            Ireland|    United States|                  ireland|          UNITED STATES|
|      United States|            Egypt|            united states|                  EGYPT|
|              India|    United States|                    india|          UNITED STATES|
|          Singapore|    United States|                singapore|          UNITED STATES|
|            Grenada|    United States|          

### 4.5 컬럼명 바꾸기

In [13]:
print("# 컬럼 명 변경하기")
df.withColumnRenamed("DEST_COUNTRY_NAME", "Destination").columns

# 컬럼 명 변경하기


['Destination', 'ORIGIN_COUNTRY_NAME', 'count']

### 4.6 컬럼 제거하기

In [14]:
print("# 특정 컬럼을 제거합니다")
df.printSchema()
df.drop("ORIGIN_COUNTRY_NAME").columns

# 특정 컬럼을 제거합니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



['DEST_COUNTRY_NAME', 'count']

In [15]:
print("# 기본적으로 스파크는 대소문자를 가리지 않지만, 옵션을 통해서 구분이 가능합니다")
spark.conf.set('spark.sql.caseSensitive', True)
caseSensitive = df.drop("dest_country_name")
caseSensitive.printSchema()

spark.conf.set('spark.sql.caseSensitive', False)
caseInsensitive = df.drop("dest_country_name")
caseInsensitive.printSchema()

# 기본적으로 스파크는 대소문자를 가리지 않지만, 옵션을 통해서 구분이 가능합니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [16]:
print("# 한 번에 여러 컬럼도 삭제할 수 있습니다")
df.printSchema()
df.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns # 여러 컬럼을 지우기

# 한 번에 여러 컬럼도 삭제할 수 있습니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



['count']

### <font color=blue>3. [중급]</font> "data/flight-data/json/2015-summary.json" 경로의 JSON 데이터를 읽고
#### 1. ORIGIN_COUNTRY_NAME 컬럼은 Origin 으로 이름을 변경하고
#### 2. DEST_COUNTRY_NAME 컬럼은 DestUpper 으로 대문자로 변경한 컬럼을 추가하고
#### 3. DEST_COUNTRY_NAME 컬럼은 Drop 하고, Origin, DestUpper 2개의 컬럼만 남은 DataFrame 의 데이터를 출력하세요
#### 4. 최종 스키마를 출력하세요

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다


```python
df3 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)
    
df3.printSchema()
answer = (df3
    .withColumnRenamed("ORIGIN_COUNTRY_NAME", "Origin")
    .withColumn("DestUpper", upper("DEST_COUNTRY_NAME"))
    .drop("DEST_COUNTRY_NAME", "count")
)
answer.show()
answer.printSchema()
```

</details>


In [17]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df3 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)

df3.printSchema()
answer = (df3
    .withColumnRenamed("ORIGIN_COUNTRY_NAME", "Origin")
    .withColumn("DestUpper", upper("DEST_COUNTRY_NAME"))
    .drop("DEST_COUNTRY_NAME", "count")
)
answer.show()
answer.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+----------------+--------------------+
|          Origin|           DestUpper|
+----------------+--------------------+
|         Romania|       UNITED STATES|
|         Croatia|       UNITED STATES|
|         Ireland|       UNITED STATES|
|   United States|               EGYPT|
|           India|       UNITED STATES|
|       Singapore|       UNITED STATES|
|         Grenada|       UNITED STATES|
|   United States|          COSTA RICA|
|   United States|             SENEGAL|
|   United States|             MOLDOVA|
|    Sint Maarten|       UNITED STATES|
|Marshall Islands|       UNITED STATES|
|   United States|              GUYANA|
|   United States|               MALTA|
|   United States|            ANGUILLA|
|   United States|             BOLIVIA|
|        Paraguay|       UNITED STATES|
|   United States|             ALGERIA|
|   United States|T

### 4.7 컬럼의 데이터 타입 변경하기

In [18]:
print("# 컬럼의 데이터 유형을 변경합니다")
df.printSchema()

int2str = df.withColumn("str_count", col("count").cast("string"))
int2str.show(5)
int2str.printSchema()

str2int = int2str.withColumn("int_count", col("str_count").cast("int"))
str2int.show(5)
str2int.printSchema()

# 컬럼의 데이터 유형을 변경합니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|str_count|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|       15|
|    United States|            Croatia|    1|        1|
|    United States|            Ireland|  344|      344|
|            Egypt|      United States|   15|       15|
|    United States|              India|   62|       62|
+-----------------+-------------------+-----+---------+
only showing top 5 rows

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- str_count: string (nullable = true)

+-----------------+-------------------+-----+---------+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|str_count|int_cou

### 4.8 레코드 필터링

In [19]:
print("# Where 와 Filter 는 동일합니다")
df.where("count < 2").show(2)
df.filter("count < 2").show(2)

print("# 같은 표현식에 여러 필터를 적용하는 것도 가능합니다")
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

# Where 와 Filter 는 동일합니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

# 같은 표현식에 여러 필터를 적용하는 것도 가능합니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



### 4.9 유일 값 (DISTINCT)

In [20]:
""" distinct 함수 """
print(df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count())
print(df.select("ORIGIN_COUNTRY_NAME").distinct().count())
# distinctcount?

256
125


### <font color=green>4. [기본] </font> "data/flight-data/json/2015-summary.json" 경로의 JSON 데이터를 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터 10건을 출력하세요
#### 3. count 가 5000 이상, 100000 보다 미만인 ORIGIN_COUNTRY_NAME 를 출력하되 중복을 제거해 주세요

<details><summary>[실습4] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다


```python
df4 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)
    
df4.printSchema()
df4.show(10)
df4.selectExpr("min(count)", "max(count)").show()
answer = df4.where(expr("count >= 5000 and count < 100000")).select("ORIGIN_COUNTRY_NAME")
answer.distinct()
```

</details>


In [21]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df4 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)

df4.printSchema()
df4.show(10)
df4.selectExpr("min(count)", "max(count)").show()
answer = df4.where(expr("count >= 5000 and count < 100000")).select("ORIGIN_COUNTRY_NAME")
answer.distinct()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows

+----------+----------+
|min(count)|max(count)|
+----------+----------+
|         1|    370002|
+----------+----------+



ORIGIN_COUNTRY_NAME
United States
Mexico
Canada


### 4.10 정렬 (SORT)

In [22]:
print("# sort 와 orderBy 함수는 동일한 효과를 가집니다")
df.sort("count").show(2)
df.orderBy("count", "DEST_COUNTRY_NAME").show(2)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(2)

# sort 와 orderBy 함수는 동일한 효과를 가집니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [23]:
from pyspark.sql.functions import *
print("# asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드로 null의 정렬 순서를 지정")
df.sort("DEST_COUNTRY_NAME").show(1)
df.sort(df["DEST_COUNTRY_NAME"].asc_nulls_first()).show(1)
df.sort(df.DEST_COUNTRY_NAME.asc_nulls_first()).show(1)

# asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드로 null의 정렬 순서를 지정
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
+-----------------+-------------------+-----+
only showing top 1 row

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
+-----------------+-------------------+-----+
only showing top 1 row

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
+-----------------+-------------------+-----+
only showing top 1 row



In [24]:
print("# 정렬의 경우 예약어 컬럼명에 유의해야 하므로, expr 을 사용하거나, 명시적으로 구조화 API 를 사용하는 것도 좋습니다") 
from pyspark.sql.functions import desc, asc
df.orderBy(df["count"].desc()).show(2)
df.orderBy(df.ORIGIN_COUNTRY_NAME.desc(), df.DEST_COUNTRY_NAME.asc()).show(2)
df.orderBy(expr("ORIGIN_COUNTRY_NAME DESC"), expr("DEST_COUNTRY_NAME ASC")).show(2)

# 정렬의 경우 예약어 컬럼명에 유의해야 하므로, expr 을 사용하거나, 명시적으로 구조화 API 를 사용하는 것도 좋습니다
+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Vietnam|    2|
|    United States|          Venezuela|  246|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|             Angola|   13|
|    United States|           Anguilla|   38|
+-----------------+-------------------+-----+
only showing top 2 rows



### 4.11 로우 수 제한 (LIMIT)

In [25]:
df.limit(5).show()
df.orderBy(expr("count desc")).limit(6).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



### <font color=red>5. [고급]</font> "data/flight-data/json/2015-summary.json" 경로의 JSON 데이터를 읽고
#### 1. 스키마를 출력하세요
#### 2. 10건의 데이터를 출력하세요
#### 3. count 를 100으로 나눈 몫을 가지는 cnt 컬럼을 추가합니다 표현식은 다음과 같습니다 `expr("floor(count / 100)")` - 힌트: withColumn("컬럼명", "표현식")
#### 4. cnt 컬럼을 기준으로 내림차순 정렬하되, 동순위가 발생하는 경우 ORIGIN_COUNTRY_NAME 오름차순, DEST_COUNTRY_NAME 내림차순으로 정렬하여 출력하세요
#### 5. 출력된 결과의 상위 10개만 제한하여 (limit) display 함수로 출력하세요

<details><summary>[실습5] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다


```python
df5 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)
    
df5.printSchema()
answer = df5.withColumn("cnt", expr("floor(count / 100)")).orderBy(desc("cnt"), asc("ORIGIN_COUNTRY_NAME"), desc("DEST_COUNTRY_NAME")).limit(10)
display(answer)
```

</details>


In [26]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df5 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .json("data/flight-data/json/2015-summary.json")
)

df5.printSchema()
answer = df5.withColumn("cnt", expr("floor(count / 100)")).orderBy(desc("cnt"), asc("ORIGIN_COUNTRY_NAME"), desc("DEST_COUNTRY_NAME")).limit(10)
display(answer)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count,cnt
United States,United States,370002,3700
United States,Canada,8483,84
Canada,United States,8399,83
United States,Mexico,7187,71
Mexico,United States,7140,71
United Kingdom,United States,2025,20
United States,United Kingdom,1970,19
Japan,United States,1548,15
United States,Dominican Republic,1420,14
United States,Japan,1496,14


## 5. 기타 데이터 프레임 연산자

### 5.1 사전에 스키마를 정의하는 장점
* 데이터 타입을 추론에 대한 신경을 쓸 필요가 없다
* 스키마 추론을 위한 별도의 작업에 드는 리소스를 줄일 수 있다
* 스키마에 맞지 않는 데이터의 오류를 빠르게 인지할 수 있다

### 5.2 스키마를 정의하는 두 가지 방법
* 1. 프로그래밍 방식으로 정의하는 방법
* 2. DDL 구문을 이용하는 방법

In [27]:
from pyspark.sql.types import *
from pyspark.sql import Row

data = [
    ["정휘센", "안녕하세요 정휘센 입니다", 300],
    ["김싸이언", "안녕하세요 김싸이언 입니다", 200],
    ["유코드제로", "안녕하세요 유코드제로 입니다", 100]
]

print("# 1. Programming Style")
schema1 = StructType([
    StructField("author", StringType(), False),
    StructField("title", StringType(), False),
    StructField("pages", IntegerType(), False),
])
print(schema1)
df1 = spark.createDataFrame(data, schema1)
df1.printSchema()
df1.show(truncate=False)

rows = [
    Row("정휘센", "안녕하세요 정휘센 입니다", 300),
    Row("김싸이언", "안녕하세요 김싸이언 입니다", 200),
    Row("유코드제로", "안녕하세요 유코드제로 입니다", 100)
]

print("\n# 2. DDL Style")
schema2 = "`author` string, `title` string, `pages` int"
print(schema2)
df2 = spark.createDataFrame(rows, schema2)
df2.printSchema()
df2.show(truncate=False)

assert(df1.subtract(df2).count() == 0)
assert(df2.subtract(df1).count() == 0)

# 1. Programming Style
StructType(List(StructField(author,StringType,false),StructField(title,StringType,false),StructField(pages,IntegerType,false)))
root
 |-- author: string (nullable = false)
 |-- title: string (nullable = false)
 |-- pages: integer (nullable = false)

+----------+----------------------------+-----+
|author    |title                       |pages|
+----------+----------------------------+-----+
|정휘센    |안녕하세요 정휘센 입니다    |300  |
|김싸이언  |안녕하세요 김싸이언 입니다  |200  |
|유코드제로|안녕하세요 유코드제로 입니다|100  |
+----------+----------------------------+-----+


# 2. DDL Style
`author` string, `title` string, `pages` int
root
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- pages: integer (nullable = true)

+----------+----------------------------+-----+
|author    |title                       |pages|
+----------+----------------------------+-----+
|정휘센    |안녕하세요 정휘센 입니다    |300  |
|김싸이언  |안녕하세요 김싸이언 입니다  |200  |
|유코드제로|안녕하세요 유코드제로 입니다|100  |
+----------+------

### <font color=red>6. [고급]</font> Row 와 문자열을 통한 스키마 구현을 통해 데이터 프레임을 생성하세요
#### 1. 스키마 : id int, name string, payment int
#### 2. 임의의 데이터를 3건 정도 생성해서 데이터프레임을 만들어 보세요
#### 3. 스키마를 출력하세요
#### 4. 데이터를 출력하세요

<details><summary>[실습6] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다


```python
df6 = [
    Row(1, "엘지전자", 1000),
    Row(2, "엘지화학", 2000),
    Row(3, "엘지디스플레이", 3000)
]
sc6 = "`id` int, `name` string, `payment` int"
answer = spark.createDataFrame(df6, sc6)
answer.printSchema()
display(answer)
```

</details>


In [28]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df6 = [
    Row(1, "엘지전자", 1000),
    Row(2, "엘지화학", 2000),
    Row(3, "엘지디스플레이", 3000)
]
sc6 = "`id` int, `name` string, `payment` int"
answer = spark.createDataFrame(df6, sc6)
answer.printSchema()
display(answer)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- payment: integer (nullable = true)



id,name,payment
1,엘지전자,1000
2,엘지화학,2000
3,엘지디스플레이,3000


### 5.3 중첩된 배열 스키마

In [29]:
schema = StructType([
    StructField("Id", IntegerType(), False),
    StructField("First", StringType(), False),
    StructField("Last", StringType(), False),
    StructField("Url", StringType(), False),
    StructField("Published", StringType(), False),
    StructField("Hits", IntegerType(), False),
    StructField("Campaigns", ArrayType(StringType()), False),
])
blogDF = spark.read.schema(schema).json("data/learning-spark/blogs.json")
blogDF.printSchema()
blogDF.show(1, truncate=False)

root
 |-- Id: integer (nullable = true)
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Published: string (nullable = true)
 |-- Hits: integer (nullable = true)
 |-- Campaigns: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---+-----+-----+-----------------+---------+----+-------------------+
|Id |First|Last |Url              |Published|Hits|Campaigns          |
+---+-----+-----+-----------------+---------+----+-------------------+
|1  |Jules|Damji|https://tinyurl.1|1/4/2016 |4535|[twitter, LinkedIn]|
+---+-----+-----+-----------------+---------+----+-------------------+
only showing top 1 row



### 5.4 컬럼과 표현식
> 컬럼은 공용 메소드들을 가진 객체들이며, pyspark.sql.functions.expr() 함수를 이용하여 표현식을 그대로 사용할 수 있습니다

* 특히 컬럼 함수를 통해 다양한 연산자를 확인할 수 있습니다.

In [30]:
from pyspark.sql.functions import Column
print(blogDF.columns)
# help(Column)

['Id', 'First', 'Last', 'Url', 'Published', 'Hits', 'Campaigns']


In [31]:
blogDF.withColumn("AuthorsId", (concat(expr("First"), lit("."), expr("Last"), lit("@"), expr("Id"))))\
.select(col("AuthorsId"))\
.show(4)

+---------------+
|      AuthorsId|
+---------------+
|  Jules.Damji@1|
| Brooke.Wenig@2|
|    Denny.Lee@3|
|Tathagata.Das@4|
+---------------+
only showing top 4 rows



In [32]:
blogDF.select(expr("Hits")).show(2)
blogDF.select(col("Hits")).show(2)
blogDF.select("Hits").show(2)

+----+
|Hits|
+----+
|4535|
|8908|
+----+
only showing top 2 rows

+----+
|Hits|
+----+
|4535|
|8908|
+----+
only showing top 2 rows

+----+
|Hits|
+----+
|4535|
|8908|
+----+
only showing top 2 rows



In [33]:
blogDF.sort(col("Id").desc()).show()

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+



### 5.5 로우 생성 및 다루기
* 로우의 경우 컬럼을 인덱스를 기준으로 접근할 수 있습니다.

In [34]:
from pyspark.sql import Row
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015",
["twitter", "LinkedIn"])
print(blog_row[1])

rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, ["Authors", "State"])
authors_df.show()

Reynold
+-------------+-----+
|      Authors|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



In [35]:
# In Python, define a schema
from pyspark.sql.types import *

# Programmatic way to define a schema
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
StructField('UnitID', StringType(), True),
StructField('IncidentNumber', IntegerType(), True),
StructField('CallType', StringType(), True),
StructField('CallDate', StringType(), True),
StructField('WatchDate', StringType(), True),
StructField('CallFinalDisposition', StringType(), True),
StructField('AvailableDtTm', StringType(), True),
StructField('Address', StringType(), True),
StructField('City', StringType(), True),
StructField('Zipcode', IntegerType(), True),
StructField('Battalion', StringType(), True),
StructField('StationArea', StringType(), True),
StructField('Box', StringType(), True),
StructField('OriginalPriority', StringType(), True),
StructField('Priority', StringType(), True),
StructField('FinalPriority', IntegerType(), True),
StructField('ALSUnit', BooleanType(), True),
StructField('CallTypeGroup', StringType(), True),
StructField('NumAlarms', IntegerType(), True),
StructField('UnitType', StringType(), True),
StructField('UnitSequenceInCallDispatch', IntegerType(), True),
StructField('FirePreventionDistrict', StringType(), True),
StructField('SupervisorDistrict', StringType(), True),
StructField('Neighborhood', StringType(), True),
StructField('Location', StringType(), True),
StructField('RowID', StringType(), True),
StructField('Delay', FloatType(), True)])

# Use the DataFrameReader interface to read a CSV file
sf_fire_file = "data/learning-spark/sf-fire-calls.csv"
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)
fire_df.select("CallNumber", "UnitID", "IncidentNumber", "CallType", "CallDate", "RowID").show(10, truncate=False)

+----------+------+--------------+----------------+----------+-------------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallDate  |RowID        |
+----------+------+--------------+----------------+----------+-------------+
|20110016  |T13   |2003235       |Structure Fire  |01/11/2002|020110016-T13|
|20110022  |M17   |2003241       |Medical Incident|01/11/2002|020110022-M17|
|20110023  |M41   |2003242       |Medical Incident|01/11/2002|020110023-M41|
|20110032  |E11   |2003250       |Vehicle Fire    |01/11/2002|020110032-E11|
|20110043  |B04   |2003259       |Alarms          |01/11/2002|020110043-B04|
|20110072  |T08   |2003279       |Structure Fire  |01/11/2002|020110072-T08|
|20110125  |E33   |2003301       |Alarms          |01/11/2002|020110125-E33|
|20110130  |E36   |2003304       |Alarms          |01/11/2002|020110130-E36|
|20110197  |E05   |2003343       |Medical Incident|01/11/2002|020110197-E05|
|20110215  |E06   |2003348       |Medical Incident|01/11/2002|020110215-E06|

### 5.6 파케이 파일 혹은 테이블 저장
* save 저장 시에는 해당 경로에 파케이 파일이 저장되고, saveAsTable 저장 시에는 "spark.sql.warehouse.dir" 의 위치에 생성됩니다

In [36]:
parquetPath="target/sf_fire_calls"
fire_df.write.format("parquet").mode("overwrite").save(parquetPath)

In [37]:
!rm -rf "/home/jovyan/work/spark-warehouse/sf_fire_calls"

In [38]:
parquetTable="sf_fire_calls"
# spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
fire_df.write.format("parquet").saveAsTable(parquetTable)

### 5.7 프로젝션과 필터
> *Projection*은 특정 관계형 조건 혹은 필터에 매칭되는 로우에 대해서만 반환하는 것을 말합니다.

In [39]:
few_fire_df = (
    fire_df
    .select("IncidentNumber", "AvailableDtTm", "CallType")
    .where(col("CallType") != "Medical Incident")
)
print(few_fire_df)
few_fire_df.show(5, truncate=False)

new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")
(
    new_fire_df
    .select("ResponseDelayedinMins")
    .where(col("ResponseDelayedinMins") > 5)
    .show(5, False)
)

+--------------+----------------------+-----------------------------+
|IncidentNumber|         AvailableDtTm|                     CallType|
+--------------+----------------------+-----------------------------+
|       2003235|01/11/2002 01:51:44 AM|               Structure Fire|
|       2003250|01/11/2002 04:16:46 AM|                 Vehicle Fire|
|       2003259|01/11/2002 06:01:58 AM|                       Alarms|
|       2003279|01/11/2002 08:03:26 AM|               Structure Fire|
|       2003301|01/11/2002 09:46:44 AM|                       Alarms|
|       2003304|01/11/2002 09:58:53 AM|                       Alarms|
|       2003382|01/11/2002 02:59:04 PM|               Structure Fire|
|       2003408|01/11/2002 04:09:08 PM|               Structure Fire|
|       2003408|01/11/2002 04:09:08 PM|               Structure Fire|
|       2003408|01/11/2002 04:09:08 PM|               Structure Fire|
|       2003429|01/11/2002 05:17:15 PM|     Odor (Strange / Unknown)|
|       2003453|01/1

### 5.8 날짜 관련 함수
* 날짜의 경우 문자열로 전달되고 있기 때문에 표현 및 활용을 위해서는 to_timestamp(), to_date() 와 같은 날짜관련 함수를 사용할 수 있습니다.
  - 한번 timestamp 형태로 변경된 컬럼에 대해서는 year, month, dayofmonth 와 같은 일자관련 함수를 통해 다양한 예제를 실습할 수 있습니다

In [40]:
fire_ts_df = (new_fire_df
    .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
    .drop("CallDate")
    .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
    .drop("WatchDate")
    .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"),"MM/dd/yyyy hh:mm:ss a"))
    .drop("AvailableDtTm")
)
fire_ts_df.select("IncidentDate", "OnWatchDate", "AvailableDtTS").show(5, truncate=False)

from pyspark.sql.functions import *
(
    fire_ts_df
    .select(year('IncidentDate'), month('IncidentDate'), dayofmonth("IncidentDate"))
    .distinct()
    .orderBy(year('IncidentDate'), month('IncidentDate'), dayofmonth("IncidentDate"))
    .show(5)
)

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows

+------------------+-------------------+------------------------+
|year(IncidentDate)|month(IncidentDate)|dayofmonth(IncidentDate)|
+------------------+-------------------+------------------------+
|              2000|                  4|                      12|
|              2000|                  4|                      13|
|              2000|                  4|                      14|
|              2000| 

### <font color=green>7. [기본]</font> 아래와 같이 제공된 데이터를 통해 생성한 데이터를 파케이 포맷으로 저장하세요
#### 1. "target/lgde_user" 경로에 파케이 파일로 저장하세요
#### 2. "lgde_user" 테이블로 저장하세요

<details><summary>[실습7] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df6 = [
    Row(1, "엘지전자", 1000),
    Row(2, "엘지화학", 2000),
    Row(3, "엘지디스플레이", 3000)
]
sc6 = "`id` int, `name` string, `payment` int"
answer = spark.createDataFrame(df6, sc6)
answer.printSchema()
display(answer)

answer.write.format("parquet").save("target/lgde_user")
answer.write.format("parquet").saveAsTable("lgde_user")
```

</details>


In [41]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df6 = [
    Row(1, "엘지전자", 1000),
    Row(2, "엘지화학", 2000),
    Row(3, "엘지디스플레이", 3000)
]
sc6 = "`id` int, `name` string, `payment` int"
answer = spark.createDataFrame(df6, sc6)
answer.printSchema()
display(answer)

answer.write.format("parquet").save("target/lgde_user")
answer.write.format("parquet").saveAsTable("lgde_user")

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- payment: integer (nullable = true)



id,name,payment
1,엘지전자,1000
2,엘지화학,2000
3,엘지디스플레이,3000


### <font color=green>8. [기본]</font> '실습7' 에서 생성한 데이터를 읽어서 출력하세요
#### 1. "target/lgde_user" 경로에 파케이 파일을 읽어서 스키마와 데이터를 출력하세요
#### 2. "lgde_user" 테이블로 저장된 테이블을 spark sql 로 읽어서 스키마와 데이터를 출력하세요

<details><summary>[실습8] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df8 = (
    spark
    .read.parquet("target/lgde_user")
)
df8.printSchema()
display(df8)

answer = spark.sql("select * from lgde_user")
display(answer)
```

</details>


In [42]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df8 = (
    spark
    .read.parquet("target/lgde_user")
)
df8.printSchema()
display(df8)

answer = spark.sql("select * from lgde_user")
display(answer)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- payment: integer (nullable = true)



id,name,payment
3,엘지디스플레이,3000
1,엘지전자,1000
2,엘지화학,2000


id,name,payment
3,엘지디스플레이,3000
1,엘지전자,1000
2,엘지화학,2000


## 6. 데이터셋 API
> Python 과 R 은 compile-time type-safe 하지 않기 때문에, Datasets 통한 Typed 데이터 타입을 사용할 수 없습니다. Datasets 을 이용하는 경우에도 Spark SQL 엔진이 객체를 생성, 변환, 직렬화, 역직렬화를 수행하며, **Dataframe 의 경우와 마찬가지로 Off-heap 을 통한 메모리 관리를 수행**하게 되며, Dataset encoders 를 이용합니다

### 6.1 데이터셋과 데이터프레임 비교

| Structured APIs | SQL vs. Dataframe vs. Datasets |
|---|---|
| ![structured-api](images/structured-api.png) | ![sql-vs-dataframes-vs-datasets-type-safety-spectrum](images/sql-vs-dataframes-vs-datasets-type-safety-spectrum.png) |

* 언어별 타입 객체 비교
![typed-untyped](images/typed-untyped.png)

* Scala: Case Class 를 통해 선언
```scala
case class DeviceIoTData (
    battery_level: Long, 
    c02_level: Long,
    cca2: String, 
    cca3: String, 
    cn: String, 
    device_id: Long,
    device_name: String, 
    humidity: Long, 
    ip: String, 
    latitude: Double,
    lcd: String, 
    longitude: Double, 
    scale:String, 
    temp: Long,
    timestamp: Long)
```

* 데이터를 읽고 DeviceIoTData 클래스로 변환을 수행합니다
```scala
val ds = spark.read.json("/databricks-datasets/learning-spark-v2/iot-devices/iot_devices.json").as[DeviceIoTData]
val filterTempDS = ds.filter({d => {d.temp > 30 && d.humidity > 70})
```
* Datasets 이용 시에는 filter(), map(), groupBy(), select(), take() 등의 일반적인 함수를 사용합니다
```scala                              
case class DeviceTempByCountry(temp: Long, device_name: String, device_id: Long, cca3: String)
val dsTemp = ds.filter(d => {d.temp > 25})
    .map(d => (d.temp, d.device_name, d.device_id, d.cca3))
    .toDF("temp", "device_name", "device_id", "cca3")
    .as[DeviceTempByCountry]
```

### 6.2 데이터셋 데이터프레임 그리고 RDD
* Datasets
  - compile-time 의 type safety 가 필요한 경우
* Dataframe
  - SQL-like 쿼리를 이용하고자 하는 경우
  - 통합, 코드 최적화 그리고 API를 활용한 모듈화를 원하는 경우
  - R 혹은 Python 을 이용해야 하는 경우
  - 공간, 속도 효율성을 고려해야 하는 경우
* RDD
  - 별도의 RDD를 이용하는 써드파티 패키지를 사용하는 경우
  - 코드, 공간, 속도 최적화 등을 원하지 않는 경우
  - 스파크가 수행할 쿼리를 정확히 지시해야만 할 때


* RDD와 데이터프레임과 데이터셋은 서로 다른가?
  - 데이터프레임과 데이터셋은 RDD 위에서 구현됩니다. 즉, whole-stage code generation 단계에서 압축된 RDD 코드로 분해됩니다.

> DataFrames and Datasets are
built on top of RDDs, and they get decomposed to compact RDD code during wholestage
code generation, which we discuss in the next section

* Spark SQL
![spark-sql](images/spark-sql.png)

## 7. 카탈리스트 옵티마이저 (참고용)
> Spark SQL 엔진의 핵심이며 크게 4가지 단계로 구분됩니다. 

### 7.1 분석 (Analysis)
* "추상화 구문 트리(AST, Abstract Syntax Tree)" 생성 단계로, 모든 테이블명과 컬럼명은 내부적으로 컬럼명, 데이터유형, 함수와 더불어 데이터베이스와 테이블 이름까지 모두 관리하고 있는 *Catalog*에 의해 해석되어 트리 형태의 구조로 생성됩니다

### 7.2 논리 최적화 (Logical Optimization)
* 카탈리스트 옵티마이저는 우선 다수의 논리적 계획을 세우고, "비용 기반 옵티마이저(CBO, Cost-Based Optimizer)"를 이용하여 각 계획에 비용(Cost)를 할당합니다. 이러한 계획은 아래의 "Figure 3-5"와 같은 연산자 트리 형태로 구성되며, 이때에 **constant folding, predicate pushdown, projection pruning, Boolean expression simplification** 등의 최적화가 이루어집니다

### 7.3 물리 계획 (Physical Planning)
* Spark SQL 엔진은 CBO에 의해 선택된 논리 계획에 대해 스파크 엔진에 존재하는 적절한 연산자들을 이용하여 최적의 계획을 생성합니다

### 7.4 코드 생성 (Code Generation)
* 마지막 단계에서는  Project Tungsten 의 whole-stage code generation 을 통해 [마치 컴파일러와 같이 동작](https://databricks.com/blog/2016/05/23/apache-spark-as-a-compiler-joining-a-billion-rows-per-second-on-a-laptop.html)하며, 메모리 상에 로딩된 데이터 집합에 대해 수행될 최적의 자바 바이트 코드를 생성해 냅니다.

* What is ***whole-stage code generation***?
  - 물리적인 쿼리 최적화 단계를 말하며, 쿼리 전체를 하나의 함수로 만들어 냅니다
  - virtual function call 을 제거하거나, 중간 데이터를 CPU registers 에 올리는 등의 최적화 작업을 수행합니다
  - Spark 2.0 텅스텐 엔진은 압축된 RDD 코드를 생성하는 방식으로 개선 되었습니다

In [43]:
mnm_df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/databricks/mnm_dataset.csv")
)
count_mnm_df = (
    mnm_df.select("State", "Color", "Count")
    .groupBy("State", "Color", "Count")
    .agg(sum("Count")
    .alias("Total"))
    .orderBy("Total", ascending=False)
)
count_mnm_df.explain(True)

== Parsed Logical Plan ==
'Sort ['Total DESC NULLS LAST], true
+- Aggregate [State#1600, Color#1601, Count#1602], [State#1600, Color#1601, Count#1602, sum(cast(Count#1602 as bigint)) AS Total#1613L]
   +- Project [State#1600, Color#1601, Count#1602]
      +- Relation[State#1600,Color#1601,Count#1602] csv

== Analyzed Logical Plan ==
State: string, Color: string, Count: int, Total: bigint
Sort [Total#1613L DESC NULLS LAST], true
+- Aggregate [State#1600, Color#1601, Count#1602], [State#1600, Color#1601, Count#1602, sum(cast(Count#1602 as bigint)) AS Total#1613L]
   +- Project [State#1600, Color#1601, Count#1602]
      +- Relation[State#1600,Color#1601,Count#1602] csv

== Optimized Logical Plan ==
Sort [Total#1613L DESC NULLS LAST], true
+- Aggregate [State#1600, Color#1601, Count#1602], [State#1600, Color#1601, Count#1602, sum(cast(Count#1602 as bigint)) AS Total#1613L]
   +- Relation[State#1600,Color#1601,Count#1602] csv

== Physical Plan ==
*(3) Sort [Total#1613L DESC NULLS LAST], tru

In [44]:
mnm_df.createOrReplaceTempView("mnm_dataset")
count_mnm_df = spark.sql("""
    SELECT State, Color, Count, sum(Count) AS Total
    FROM mnm_dataset
    GROUP BY State, Color, Count
    ORDER BY Total DESC
""")
count_mnm_df.explain(True)

== Parsed Logical Plan ==
'Sort ['Total DESC NULLS LAST], true
+- 'Aggregate ['State, 'Color, 'Count], ['State, 'Color, 'Count, 'sum('Count) AS Total#1620]
   +- 'UnresolvedRelation [mnm_dataset]

== Analyzed Logical Plan ==
State: string, Color: string, Count: int, Total: bigint
Sort [Total#1620L DESC NULLS LAST], true
+- Aggregate [State#1600, Color#1601, Count#1602], [State#1600, Color#1601, Count#1602, sum(cast(Count#1602 as bigint)) AS Total#1620L]
   +- SubqueryAlias mnm_dataset
      +- Relation[State#1600,Color#1601,Count#1602] csv

== Optimized Logical Plan ==
Sort [Total#1620L DESC NULLS LAST], true
+- Aggregate [State#1600, Color#1601, Count#1602], [State#1600, Color#1601, Count#1602, sum(cast(Count#1602 as bigint)) AS Total#1620L]
   +- Relation[State#1600,Color#1601,Count#1602] csv

== Physical Plan ==
*(3) Sort [Total#1620L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(Total#1620L DESC NULLS LAST, 200), true, [id=#1170]
   +- *(2) HashAggregate(keys=[State#1600,

* 아래와 같이 2개의 테이블에 대해 조인, 필터, 프로젝션 등의 연산 시에 아래와 같은 최적화로 **Disk 및 Network I/O 를 줄일 수 있습니다**.
  - Predicate Pushdown : 데이터 소스를 모두 읽지 않고, 필터 조건에 해당하는 데이터만 읽습니다
  - Column Pruning : 데이터 소스에서 모든 필드를 읽지 않고, 필요한 필터만 읽습니다

```scala
val users = spark.read.parquet("/users/parquet/path")
val events = spark.read.parquet("/events/parquet/path")
val joinedDF = users.join(events, users("id") === events("uid"))
.filter(events("date") > "2015-01-01")
```

![query-transformation](images/query-transformation.png)

### <font color=blue>9. [중급]</font> "data/tbl_user.csv" 에 저장된 CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터를 10건 출력하세요
#### 3. 가입일자 컬럼(u_signup)을 이용하여 가장 최근에 가입한 5명을 출력하세요

<details><summary>[실습9] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df9 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/tbl_user.csv")
)
df9.printSchema()
df9.show(10)
df9.createOrReplaceTempView("tbl_user")
answer = spark.sql("select * from tbl_user order by u_signup desc limit 5")
display(answer)
```

</details>


In [47]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df9 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/tbl_user.csv")
)
df9.printSchema()
df9.show(10)
df9.createOrReplaceTempView("tbl_user")
answer = spark.sql("select * from tbl_user order by u_signup desc limit 5")
display(answer)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

+----+----------+--------+--------+
|u_id|    u_name|u_gender|u_signup|
+----+----------+--------+--------+
|   1|    정휘센|      남|19700808|
|   2|  김싸이언|      남|19710201|
|   3|    박트롬|      여|19951030|
|   4|    청소기|      남|19770329|
|   5|유코드제로|      여|20021029|
|   6|  윤디오스|      남|20040101|
|   7|  임모바일|      남|20040807|
|   8|  조노트북|      여|20161201|
|   9|  최컴퓨터|      남|20201124|
+----+----------+--------+--------+



u_id,u_name,u_gender,u_signup
9,최컴퓨터,남,20201124
8,조노트북,여,20161201
7,임모바일,남,20040807
6,윤디오스,남,20040101
5,유코드제로,여,20021029


### <font color=blue>10. [중급]</font> "data/tbl_purchase.csv" 에 저장된 CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터를 10건 출력하세요
#### 3. 상품가격 컬럼(p_amount)을 이용하여 200만원 이상 금액의 상품 가운데 상위 3개를 출력하세요

<details><summary>[실습10] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df10 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/tbl_purchase.csv")
)
df10.printSchema()
df10.show(10)
df10.createOrReplaceTempView("tbl_purchase")
answer = spark.sql("select * from tbl_purchase where p_amount > 2000000 order by p_amount desc limit 3")
display(answer)
```

</details>


In [48]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
df10 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/tbl_purchase.csv")
)
df10.printSchema()
df10.show(10)
df10.createOrReplaceTempView("tbl_purchase")
answer = spark.sql("select * from tbl_purchase where p_amount > 2000000 order by p_amount desc limit 3")
display(answer)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

+----------+-----+----+-----------+--------+
|    p_time|p_uid|p_id|     p_name|p_amount|
+----------+-----+----+-----------+--------+
|1603651550|    0|1000|GoldStar TV|  100000|
|1603651550|    1|2000|    LG DIOS| 2000000|
|1603694755|    1|2001|    LG Gram| 1800000|
|1603673500|    2|2002|    LG Cyon| 1400000|
|1603652155|    3|2003|      LG TV| 1000000|
|1603674500|    4|2004|LG Computer| 4500000|
|1603665955|    5|2001|    LG Gram| 3500000|
|1603666155|    5|2003|      LG TV| 2500000|
+----------+-----+----+-----------+--------+



p_time,p_uid,p_id,p_name,p_amount
1603674500,4,2004,LG Computer,4500000
1603665955,5,2001,LG Gram,3500000
1603666155,5,2003,LG TV,2500000


## 참고자료

#### 1. [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
#### 2. [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
#### 3. <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>